In [1]:
import numpy as np 
from random import random
import cv2 as cv

In [2]:
cap = cv.VideoCapture('manas_final_task_dataset.mp4')

In [3]:
type(cap)

cv2.VideoCapture

In [4]:
labelsPath = 'yolo-coco\\coco.names'
LABELS = open(labelsPath).read().strip().split('\n')

COLORS = np.random.randint(0,255, size=(len(LABELS), 3), dtype= "uint8")

In [5]:
def vsum(v1,v2):
    return (int(v1[0]+v2[0]),int(v1[1]+v2[1]))

def dot(v1,v2):
    return v1[0]*v2[0]+v1[1]*v2[1]

def norm(v):
    return dot(v,v)**0.5

def normalize(v):
    if norm(v) == 0:
        return v
    return (v[0]/norm(v), v[1]/norm(v))

def mult(M, v):
    Mv = ()
    for row in M:
        Mv += (dot(row,v),)
    return Mv

def setMag(v,m):
    return scale(normalize(v), m)

def vdiff(v1,v2):
    return (v1[0]-v2[0],v1[1]-v2[1])

def scale(v,r):
    return r*v[0], r*v[1]

def dist(p1, p2):
    return round((int((p2[0]-p1[0])**2) + int((p2[1]-p1[1])**2))**0.5, 2)

In [6]:
weightsPath = 'yolo-coco\\yolov3.weights' 
configPath = 'yolo-coco\\yolov3.cfg'
net = cv.dnn.readNet(weightsPath, configPath)

In [7]:
def get_layer_names(net):
    lnames = net.getLayerNames()
    output_layers = [lnames[i-1] for i in net.getUnconnectedOutLayers()]
    return output_layers

def draw_box(frame, class_id, confidence, x, y, x_plus_w, y_plus_h):
    label = str(LABELS[class_id])
    color = tuple(COLORS[class_id])
    color = (int(color[0]),int(color[1]),int(color[2]))
    cv.rectangle(frame, (int(x),int(y)), (int(x_plus_w),int(y_plus_h)),color , 2)
    cv.putText(frame, label, (x-10, y-10),cv.FONT_HERSHEY_SIMPLEX, 0.5,color, 2)

def get_boxes(frame, conf_threshold = 0.5, nms_threshold = 0.4):
    (H,W) = frame.shape[:2]
    class_ids = []
    confidences = []
    boxes = []
    blob = cv.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    outs = net.forward(get_layer_names(net))
    
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            
            if confidence>conf_threshold:
                centerx = int(detection[0] * W)
                centery = int(detection[1] * H)
                w = int(detection[2] * W)
                h = int(detection[3] * H)
                x = centerx - w / 2
                y = centery - h / 2
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])
                
    nnms = cv.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
    return boxes, nnms, class_ids, confidences
    
def prev_box(box, boxes):
    pbox = sorted(boxes, key=lambda b: dist(box, b))[0]
    if dist(pbox, box)> 50*box[2]/100:
        return box
    return pbox
    
def predict(p, r):
    prediction = setMag(vdiff(r, p), 50)
    
    return prediction

def trajectory(box, object_history):
    cbox = box
    traj = []
    for boxes in object_history:
        pbox = prev_box(cbox, boxes)
        traj.append(pbox)
        cbox = pbox
        
    return traj[::-1]

def evaluate(p, x):
    return sum([p[i]*x**i for i in range(len(p))])

def PR(pts,deg,w,h,f):
    n = len(pts)
    xs,ys = [pt[0] for pt in pts], [pt[1] for pt in pts]
    T = np.array([[(i+1)**k for k in range(deg+1)] for i in range(len(pts))])
    x = np.transpose(np.array(xs))
    y = np.transpose(np.array(ys))
    Tt = np.transpose(T)
    A = np.matmul(Tt,T)
    alpha = np.matmul(np.linalg.inv(A), np.matmul(Tt,x))
    beta  = np.matmul(np.linalg.inv(A), np.matmul(Tt,y))
    point = (int(evaluate(alpha, -f)), int(evaluate(beta, -f)))
    return point

In [8]:
conf_threshold =0.5
nms_threshold = 0.3

past_frames =10 #Number of frames to be taken from past for vector calculation
object_history = [[] for i in range(past_frames)]

for i in range(past_frames-1,-1,-1):
    ret, prev_frame = cap.read()
    object_history[i],prev_nnms, pclass_id, pconfidences = get_boxes(prev_frame, conf_threshold, nms_threshold)

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        print(" Exiting ...")
        break

    (H,W) = frame.shape[:2]
    
    boxes, nnms, class_ids, confidences = get_boxes(frame,conf_threshold ,nms_threshold)

    for i in nnms: 
        box = boxes[i]
        x,y,w,h = tuple(box)
        center = (int(x+w//2),int(y+h//2))
        color = tuple(COLORS[class_ids[i]])
        color = (int(color[0]),int(color[1]),int(color[2]))
        
        traj = trajectory(box, object_history)
        if len(traj)>6:
            prediction = PR(traj,1,w,h,200)
            prediction = (prediction[0]+w//2,prediction[1]+h//2)
            diff = vdiff(prediction, center)
            #diff = setMag(diff, -h//2)
            diff = setMag(diff, -150)
            prediction = vsum(diff, center)
            #cv.line(frame, center, prediction, color, 2)
            cv.arrowedLine(frame, center, prediction, color, 2, 8, 0, 0.3 )
        
        for point in traj:
            center = (int(point[0]+point[2]//2), int(point[1]+point[3]//2))
            cv.circle(frame, center, 3, color, 3)

        draw_box(frame, class_ids[i], confidences[i], int(x), int(y), int(x+w),int(y+h))
    
    cv.imshow("Manas final task",frame)
    
    object_history.pop()
    object_history = [boxes] + object_history

    if cv.waitKey(1) == ord('q'):
        break
        
cap.release()
cv.destroyAllWindows()